In [3]:
import subprocess
import json
import snscrape

In [8]:
# Define the hashtag and timeframe
hashtag = "fitspo"
language = 'en'
place = 'usa'
start_date = "2019-01-01"
end_date = "2020-02-28"

### most recent

In [10]:
# Define the snscrape command
command = f'snscrape --jsonl --max-results 100 twitter-hashtag "{hashtag} lang:{language} place:{place} since:{start_date} until:{end_date}"'

# Run the snscrape command and capture the output
output = subprocess.check_output(command, shell=True)

# Decode the JSON output
tweets = [json.loads(line) for line in output.splitlines()]

In [6]:
for t in tweets:
    print(t['date'])

2020-02-27T23:52:16+00:00
2020-02-27T23:38:13+00:00
2020-02-27T23:29:49+00:00
2020-02-27T23:28:59+00:00
2020-02-27T23:13:19+00:00
2020-02-27T23:12:45+00:00
2020-02-27T23:04:26+00:00
2020-02-27T23:04:07+00:00
2020-02-27T23:03:24+00:00
2020-02-27T22:57:15+00:00
2020-02-27T22:50:55+00:00
2020-02-27T22:43:03+00:00
2020-02-27T22:34:19+00:00
2020-02-27T22:32:35+00:00
2020-02-27T22:15:01+00:00
2020-02-27T22:08:54+00:00
2020-02-27T22:02:29+00:00
2020-02-27T21:58:59+00:00
2020-02-27T21:58:53+00:00
2020-02-27T21:58:53+00:00
2020-02-27T21:50:42+00:00
2020-02-27T21:35:55+00:00
2020-02-27T21:34:11+00:00
2020-02-27T21:30:23+00:00
2020-02-27T21:28:53+00:00
2020-02-27T21:24:37+00:00
2020-02-27T21:21:29+00:00
2020-02-27T21:16:07+00:00
2020-02-27T21:12:35+00:00
2020-02-27T20:53:34+00:00
2020-02-27T20:40:45+00:00
2020-02-27T20:39:20+00:00
2020-02-27T20:36:34+00:00
2020-02-27T20:30:50+00:00
2020-02-27T20:30:04+00:00
2020-02-27T20:26:00+00:00
2020-02-27T19:54:48+00:00
2020-02-27T19:53:27+00:00
2020-02-27T1

In [9]:
tweets[0]

{'_type': 'snscrape.modules.twitter.Tweet',
 'url': 'https://twitter.com/MyLifeinZen1/status/1476702314899542031',
 'date': '2021-12-30T23:50:42+00:00',
 'rawContent': '#healthygirl #fitspo 25 Colors Professional Shimmer Palette https://t.co/WvmoXEobGL https://t.co/p0SlxDiozn',
 'renderedContent': '#healthygirl #fitspo 25 Colors Professional Shimmer Palette mylifeinzen.com/25-colors-prof… https://t.co/p0SlxDiozn',
 'id': 1476702314899542031,
 'user': {'_type': 'snscrape.modules.twitter.User',
  'username': 'MyLifeinZen1',
  'id': 1185107601652174849,
  'displayname': 'My Life in Zen',
  'rawDescription': 'Great selection of Health & Beauty Supplies at affordable prices! Free shipping to 185 countries. 45 days money back guarantee. Friendly customer service.',
  'renderedDescription': 'Great selection of Health & Beauty Supplies at affordable prices! Free shipping to 185 countries. 45 days money back guarantee. Friendly customer service.',
  'descriptionLinks': None,
  'verified': False

In [11]:
tweets[0]['url']

'https://twitter.com/MyLifeinZen1/status/1476702314899542031'

In [12]:
tweets[0]['rawContent']

'#healthygirl #fitspo 25 Colors Professional Shimmer Palette https://t.co/WvmoXEobGL https://t.co/p0SlxDiozn'

In [10]:
tweets[0]['date']

'2021-12-30T23:50:42+00:00'

In [15]:
tweets[0]['id']

1476702314899542031

In [16]:
tweets[0]['user']

{'_type': 'snscrape.modules.twitter.User',
 'username': 'MyLifeinZen1',
 'id': 1185107601652174849,
 'displayname': 'My Life in Zen',
 'rawDescription': 'Great selection of Health & Beauty Supplies at affordable prices! Free shipping to 185 countries. 45 days money back guarantee. Friendly customer service.',
 'renderedDescription': 'Great selection of Health & Beauty Supplies at affordable prices! Free shipping to 185 countries. 45 days money back guarantee. Friendly customer service.',
 'descriptionLinks': None,
 'verified': False,
 'created': '2019-10-18T08:17:33+00:00',
 'followersCount': 112,
 'friendsCount': 43,
 'statusesCount': 21534,
 'favouritesCount': 1,
 'listedCount': 1,
 'mediaCount': 21534,
 'location': '',
 'protected': False,
 'link': {'_type': 'snscrape.modules.twitter.TextLink',
  'text': 'mylifeinzen.com',
  'url': 'https://mylifeinzen.com/',
  'tcourl': 'https://t.co/eLy9hrqtnQ',
  'indices': [0, 23]},
 'profileImageUrl': 'https://pbs.twimg.com/profile_images/11915

In [17]:
tweets[0]['user']['id']

1185107601652174849

In [14]:
tweets[0]['user']['followersCount']

112

In [18]:
tweets[0]['user']['description']

'Great selection of Health & Beauty Supplies at affordable prices! Free shipping to 185 countries. 45 days money back guarantee. Friendly customer service.'

In [ ]:
# Print the text of each tweet
for tweet in tweets:
    print(tweet['content']['text'])